In [1]:
import numpy as np
import pandas as pd
import nltk

In [2]:
amazon_df = pd.read_csv("amazon_product.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'amazon_product.csv'

In [3]:
daraz_df = pd.read_csv(r'D:\amazon_product.csv', encoding='ISO-8859-1')

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\amazon_product.csv'

In [5]:
amazon_df = pd.read_csv(r'D:\amazon_product.csv', encoding='ISO-8859-1')

In [6]:
amazon_df.head()

,id,Title,Description,Category
0,1,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...
1,2,Gemini200 Delta CV-880 Gold Crown Livery Airc...,Welcome to the exciting world of GeminiJets! O...,Toys & Games Hobbies Models & Model Kits Pre-...
2,5,Superior Threads 10501-2172 Magnifico Cream P...,"For quilting and embroidery, this product is m...","Arts, Crafts & Sewing Sewing Thread & Floss S..."
3,6,Fashion Angels Color Rox Hair Chox Kit,Experiment with the haute trend of hair chalki...,Beauty & Personal Care Hair Care Hair Colorin...
4,8,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games âº Action Figures & Statues âº...


In [11]:

amazon_df.head()

,Title,Description,Category
0,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...
1,Gemini200 Delta CV-880 Gold Crown Livery Airc...,Welcome to the exciting world of GeminiJets! O...,Toys & Games Hobbies Models & Model Kits Pre-...
2,Superior Threads 10501-2172 Magnifico Cream P...,"For quilting and embroidery, this product is m...","Arts, Crafts & Sewing Sewing Thread & Floss S..."
3,Fashion Angels Color Rox Hair Chox Kit,Experiment with the haute trend of hair chalki...,Beauty & Personal Care Hair Care Hair Colorin...
4,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games âº Action Figures & Statues âº...


In [12]:
amazon_df.isnull().sum()

Title          0
Description    0
Category       0
dtype: int64

In [17]:
import nltk

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [19]:
nltk.word_tokenize('how are you brother.')

['how', 'are', 'you', 'brother', '.']

In [21]:
nltk.sent_tokenize('how are you. I am good. What about you.')

['how are you.', 'I am good.', 'What about you.']

In [24]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
def tokenize_stem(text):
    tokens = nltk.word_tokenize(text.lower())
    stemmed = [stemmer.stem(w) for w in tokens]
    return " ".join(stemmed)

In [27]:
amazon_df['stemmed_tokens'] = amazon_df.apply(lambda row: tokenize_stem(row["Title"] + " " + row["Description"]), axis=1)

In [28]:
amazon_df.head()

,Title,Description,Category,stemmed_tokens
0,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...,swissmar capstor select storag rack for 18-pac...
1,Gemini200 Delta CV-880 Gold Crown Livery Airc...,Welcome to the exciting world of GeminiJets! O...,Toys & Games Hobbies Models & Model Kits Pre-...,gemini200 delta cv-880 gold crown liveri aircr...
2,Superior Threads 10501-2172 Magnifico Cream P...,"For quilting and embroidery, this product is m...","Arts, Crafts & Sewing Sewing Thread & Floss S...",superior thread 10501-2172 magnifico cream puf...
3,Fashion Angels Color Rox Hair Chox Kit,Experiment with the haute trend of hair chalki...,Beauty & Personal Care Hair Care Hair Colorin...,fashion angel color rox hair chox kit experi w...
4,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games âº Action Figures & Statues âº...,union creativ giant kill figur 05 : daisuk tsu...


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create a TfidfVectorizer with the tokenize_stem function
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize_stem)

def cosine_sim(txt1, txt2):
    # Use fit_transform to create the TF-IDF matrix
    matrix = tfidf_vectorizer.fit_transform([txt1, txt2])
    
    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(matrix)
    
    # Return the cosine similarity value for the two texts
    return similarity_matrix[0, 1]


In [39]:
def recommend_product(query, df):
    # Tokenize and stem the input query
    tokenized_query = tokenize_stem(query)

    # Calculate cosine similarity between the query and each product in the dataset
    df['similarity'] = df['stemmed_tokens'].apply(lambda x: cosine_sim(tokenized_query, x))

    # Sort the DataFrame based on similarity in descending order and take the top 10 results
    final_df = df.sort_values(by=['similarity'], ascending=False).head(10)

    # Select specific columns ('Title', 'Description', 'Category') for the final recommendation
    final_df = final_df[['Title', 'Description', 'Category']]

    # Return the DataFrame with the top 10 recommended products
    return final_df


In [36]:
amazon_df['Title'][5]

' Tape Logic Shipping & Handling Label, Legend"FRAGILE - GLASS - HANDLE WITH CARE" (DL1570) '

In [40]:
recommend_product('Tape Logic Shipping & Handling Label, Legend"FRAGILE - GLASS - HANDLE WITH CARE', amazon_df)

,Title,Description,Category
5,"Tape Logic Shipping & Handling Label, Legend""...",Aviditi Tape Logic shipping label. Boldly disp...,Industrial & Scientific âº Packaging & Shipp...
89,"Libbey Vina Red Wine Glasses, Set of 6",The Libbey Vina 6-piece Red Wine Glass Set giv...,Home & Kitchen âº Kitchen & Dining âº Dinin...
500,UNIQUEPACKING 100 Pcs 4x6 Inches Clear Reseal...,Impress your customers with a unique crystal c...,"Arts, Crafts & Sewing âº Crafting âº Paper ..."
529,"Deco 79 80327 Metal Charger plate 13"" D",Elevate your table presentation with this roun...,Home & Kitchen âº Kitchen & Dining âº Dinin...
600,Pavilion- Brown Leather Flower Headband,"LAYLA is a collection of leather belts, bracel...","Clothing, Shoes & Jewelry Women Accessories H..."
455,"Pendaflex 43-1/2 Hanging File Folder Tabs, 1/...",Add a colorful touch to your organization with...,Office Products âº Office & School Supplies ...
126,"Baci Magic Colors Eyelashes, No. 525 Blue-Gli...","These are long, glamorous, diversified, hand-w...",Beauty & Personal Care Tools & Accessories Ma...
136,"Beistle 55180-CNW 12-Piece Pageant Garlands, ...",Beistle 55180-CNW Include 12 Pageant Garlands....,Home & Kitchen âº Event & Party Supplies âº...
176,Angel Soft Professional Series Premium 2-Ply ...,Angel Soft Professional Series offers a qualit...,Health & Household Household Supplies Paper &...
342,Majestic Giftware SC-CP44 Safed Handcrafted H...,"From the ancient city of Safed, Israel nested ...",Home & Kitchen Home DÃ©cor Candles & Holders ...


In [41]:
amazon_df['Title'][7]

' Frymaster 8262183 Fp28/36 Sink Drain Struct Kit '

In [44]:
recommend_product(' Frymaster 8262183 Fp28/36 Sink Drain Struct Kit ', amazon_df)

,Title,Description,Category
7,Frymaster 8262183 Fp28/36 Sink Drain Struct Kit,"826-2183, KIT, FP28/36 SINK DRAIN. Frymaster G...",Appliances Parts & Accessories Cooktop Parts ...
4,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games âº Action Figures & Statues âº...
482,Arthur Imaging Compatible Ink Cartridge Repla...,Arthur Imaging Compatible Ink Cartridge Replac...,Office Products âº Office & School Supplies ...
611,Mini-SAS Adapter - Dual SFF-8643 to SFF-8644 ...,This Mini-SAS adapter lets you convert two int...,Electronics âº Accessories & Supplies âº Au...
518,Ancor Marine Grade Heat Shrink and Nylon Disc...,Adhesive Lined Heat Shrink DisconnectsPart #: ...,Industrial & Scientific âº Industrial Electr...
302,ChiaoGoo 32-Inch Red Lace Stainless Steel Cir...,Chiaogoo red lace circular knitting needles ar...,"Arts, Crafts & Sewing âº Knitting & Crochet ..."
638,Traxxas 6433 Splitter Extension Model Car Parts,"Splitter extension / canards, left and right /...",Toys & Games âº Hobbies âº Remote & App Con...
190,"NOVICA 83062"" A Heart Shared by Two Wood Scul...","NOVICA, in association with National Geographi...",Home & Kitchen Home DÃ©cor Home DÃ©cor Accent...
421,"BeadSmith WCR-4102 11.8m Aluminum Craft Wire,...",Aluminum craft wire. Another colorful option f...,"Arts, Crafts & Sewing Beading & Jewelry Makin..."
149,Spy Gear - Ninja Attack - Ninja Stars,Conduct spy missions with the Spy Gear Ninja A...,Toys & Games âº Novelty & Gag Toys âº Spy G...
